### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [91]:
import pandas as pd
import requests
from shapely.geometry import Point
import geopandas as gpd
import re

In [92]:
interest_url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'


In [122]:
def interest_points(url):
    base_url = 'https://datos.madrid.es/egob'
    df = pd.json_normalize(requests.get(base_url + url).json()['@graph'])
    print('Interest points dataframe created')
    return df

In [125]:
interest_df = interest_points(interest_url)
interest_df.head(10)

Interest points dataframe created


,@id,id,title,relation,references,address.district.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.organization-name,address.area.@id
0,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,JARDIN TRES CULTURAS,40.464322,-3.603558,"La obra, creada por el escultor judío de orige...",A las víctimas del Holocausto,NaN
1,https://datos.madrid.es/egob/catalogo/tipo/mon...,409512,A los Abuelos,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Alicún,40.479812,-3.661719,"Pequeña escultura, situada en un parque delimi...",A los Abuelos,https://datos.madrid.es/egob/kos/Provincia/Mad...
2,https://datos.madrid.es/egob/catalogo/tipo/mon...,408781,A Neruda-El Ser alado,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Gta José Luis Ozores 116,40.381855,-3.652419,Monumento dedicado al escritor Pablo Neruda (1...,A Neruda-El Ser alado,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,https://datos.madrid.es/egob/catalogo/tipo/mon...,408836,Abogados Laboristas,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Atocha 55,40.412911,-3.700017,El 24 de enero de 1977 se produjo un atentado ...,Abogados Laboristas,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,https://datos.madrid.es/egob/catalogo/tipo/mon...,408920,Abstracta I,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,,40.421746,-3.662293,"La histórica quinta de la Fuente del Berro, de...",Abstracta I,https://datos.madrid.es/egob/kos/Provincia/Mad...
5,https://datos.madrid.es/egob/catalogo/tipo/mon...,408921,Abstracta II,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,,40.421962,-3.662260,"La histórica quinta de la Fuente del Berro, de...",Abstracta II,https://datos.madrid.es/egob/kos/Provincia/Mad...
6,https://datos.madrid.es/egob/catalogo/tipo/mon...,400147,Abundancia,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28003,CALLE GUZMAN EL BUENO 137,40.444201,-3.712595,En el año 1958 se celebró un sorteo extraordin...,Abundancia,https://datos.madrid.es/egob/kos/Provincia/Mad...
7,https://datos.madrid.es/egob/catalogo/tipo/mon...,414012,Academia de Farmacia,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Farmacia 9,40.423967,-3.699719,La Academia de Farmacia tuvo como antecedente ...,Academia de Farmacia,https://datos.madrid.es/egob/kos/Provincia/Mad...
8,https://datos.madrid.es/egob/catalogo/tipo/mon...,409550,Acueducto y rejas sobre el arroyo del Barranco...,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,CAMINO HUMERA,40.411744,-3.778890,"Esta construcción, híbrida de puente-acueducto...",Acueducto y rejas sobre el arroyo del Barranco...,NaN
9,https://datos.madrid.es/egob/catalogo/tipo/mon...,400108,Ad Iesum per Mariam,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28043,AVENIDA MACHUPICHU 1,40.455453,-3.623525,El grupo escultórico c

In [119]:
interest_df['Type of place'] = 'Monument'
interest_df=interest_df[['title', 'Type of place', 'address.street-address']]
interest_df.columns = ['Place of interest', 'Type of place', 'Place address']
interest_df

KeyError: "['title', 'address.street-address'] not in index"

In [95]:
bicimad_df = pd.read_csv('../data/origin/bicimad_stations.csv', sep='\t')
bicimad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            264 non-null    int64 
 1   id                    264 non-null    int64 
 2   name                  264 non-null    object
 3   light                 264 non-null    int64 
 4   number                264 non-null    object
 5   address               264 non-null    object
 6   activate              264 non-null    int64 
 7   no_available          264 non-null    int64 
 8   total_bases           264 non-null    int64 
 9   dock_bikes            264 non-null    int64 
 10  free_bases            264 non-null    int64 
 11  reservations_count    264 non-null    int64 
 12  geometry.type         264 non-null    object
 13  geometry.coordinates  264 non-null    object
dtypes: int64(9), object(5)
memory usage: 29.0+ KB


In [96]:
bicipark_df = pd.read_csv('../data/origin/bicipark_stations.csv', sep=';')
bicipark_df.loc[2,['geometry.coordinates']]

geometry.coordinates    [-3.690786, 40.418146]
Name: 2, dtype: object

In [98]:
bicipark_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            11 non-null     int64 
 1   stationId             11 non-null     int64 
 2   stationName           11 non-null     object
 3   address               11 non-null     object
 4   city                  11 non-null     object
 5   zip_code              11 non-null     object
 6   total_places          11 non-null     int64 
 7   free_places           11 non-null     int64 
 8   enabled               11 non-null     int64 
 9   reserved_places       11 non-null     int64 
 10  geometry.type         11 non-null     object
 11  geometry.coordinates  11 non-null     object
dtypes: int64(6), object(6)
memory usage: 1.2+ KB


In [116]:
bicimad_df[['longitude', 'latitude']] = bicimad_df['geometry.coordinates'].str.strip('[]').str.split(',',expand= True).astype('float64')
bicimad_df['longitude'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 264 entries, 0 to 263
Series name: longitude
Non-Null Count  Dtype  
--------------  -----  
264 non-null    float64
dtypes: float64(1)
memory usage: 2.2 KB


In [113]:
bicimad_df.columns

Index(['Unnamed: 0', 'id', 'name', 'light', 'number', 'address', 'activate',
       'no_available', 'total_bases', 'dock_bikes', 'free_bases',
       'reservations_count', 'geometry.type', 'geometry.coordinates'],
      dtype='object')

In [ ]:
bicimad_df.columns = ['BiciMAD station', 'light', 'number', 'Station location', 'activate',
       'no_available', 'total_bases', 'dock_bikes', 'free_bases',
       'reservations_count', 'geometry.type', 'geometry.coordinates']

In [ ]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)


interest_df.loc[:1,['stations_distances']] = interest_df.apply(lambda x: [distance_meters(x['location.latitude'],x['location.longitude'],bicimad_df.loc[i,'latitude'],bicimad_df.loc[i,'longitude']) for i in range(len(bicimad_df['longitude']))], axis=1)

KeyboardInterrupt: 

In [ ]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

# Create a helper function to calculate distance between two coordinates
def calculate_distance(row):
    lat1, lon1 = row['location.latitude'], row['location.longitude']
    distances = [distance_meters(lat1, lon1, lat2, lon2) for lat2, lon2 in zip(stations_df['latitude'], stations_df['longitude'])]
    return distances[:5]  # Only return the first 5 distances

# Apply the helper function to the dataframe
interest_df['stations_distances'] = interest_df.apply(calculate_distance, axis=1)

KeyboardInterrupt: 

In [120]:
def nearest(x):
    dist = [distance_meters(interest_df['location.latitude'][x],interest_df['location.longitude'][x] , bicimad_df['latitude'][i], bicimad_df['longitude'][i])[0] for i in range(len(bicimad_df['longitude']))]
    print(len(dist))
    return f'Your nearest station is {bicimad_df['name'][dist.index(min(dist))].split('- ')[1]} located at {round(min(dist),2)} meters from you'
      


In [124]:
nearest(5)

264


'Your nearest station is Quinta Fuente del Berro located at 356.93 meters from you'